Kaggle Challenge - Fraud Detection - BGU 2019

(1) Load Data

In [31]:
from utils import *
dataset = pd.read_csv('../Detecting-Cyber-Attacks-Kaggle-Callenge/data/partial_labels.csv')
x,segments = dataset.axes
array = dataset.values
results_df = pd.DataFrame(columns=["id","label"])

(2) Pre-Processing

In [32]:
z = np.reshape(array[0,1:],-1)
Y_train = z[0:50]
Y_train = Y_train.astype(int)
fileName = array[0,0]
path_read = "../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/User0"
all_instructions = pd.read_csv(path_read, sep=" ", header=None)
all_instructions = np.reshape(all_instructions.values,(150,100))

for testUser in range(1,40):
    print("testUser = {}".format(testUser))
    z = np.reshape(array[testUser,1:],-1)
    Y_train = z[0:50]
    Y_train = Y_train.astype(int)
    #read instructions
    fileName = array[testUser,0]
    path_read = "../Detecting-Cyber-Attacks-Kaggle-Callenge/data/FraudedRawData/User"+str(testUser)
    instructions = pd.read_csv(path_read, sep=" ", header=None)
    arr = np.reshape(instructions.values,(150,100))
    print("arr = {}".format(arr))
    print("arr.shape = {}".format(arr.shape))
    all_instructions = np.concatenate((all_instructions, arr),axis=0)

testUser = 1
arr = [['cpp' 'sh' 'xrdb' ... 'MediaMai' 'sendmail' 'hostname']
 ['id' 'nawk' 'getopt' ... 'grep' 'date' 'lp']
 ['find' 'mkdir' 'expr' ... 'find' 'sendmail' 'sendmail']
 ...
 ['netscape' 'netscape' 'hostname' ... 'netscape' 'netscape' 'hostname']
 ['id' 'nawk' 'getopt' ... 'generic' 'lp' 'getpgrp']
 ['LOCK' 'true' 'ls' ... 'sed' 'FIFO' 'rm']]
arr.shape = (150, 100)
testUser = 2
arr = [['cpp' 'sh' 'xrdb' ... 'rlogin' 'ksh' 'xterm']
 ['whoami' 'hostname' 'ksh' ... 'xterm' 'whoami' 'hostname']
 ['ksh' 'whoami' 'hostname' ... 'ls' 'ls' 'ls']
 ...
 ['netscape' 'netscape' 'netscape' ... 'popper' 'popper' 'popper']
 ['awk' 'cat' 'post' ... 'date' 'generic' 'rm']
 ['popper' 'netscape' 'popper' ... 'hostname' 'id' 'nawk']]
arr.shape = (150, 100)
testUser = 3
arr = [['touch' 'sh' 'cpp' ... 'p' 'sh' 'p']
 ['sh' 'tcpostio' 'tcpostio' ... 'rm' 'UNLOCK' 'rmdir']
 ['generic' 'tcppost' 'sh' ... 'generic' 'download' 'gethost']
 ...
 ['generic' 'ln' 'ln' ... 'netscape' 'gethost' 'movemail']

arr = [['cat' 'hostname' 'cat' ... 'ksh' 'ksh' 'ls']
 ['ls' 'ls' 'ls' ... '.xsessio' 'wait4wm' 'reaper']
 ['hostname' 'which' 'which' ... 'gettxt' 'gettxt' 'gettxt']
 ...
 ['configur' 'cat' 'configur' ... 'configur' 'cat' 'configur']
 ['fec' 'be' 'ld_' ... 'sed' 'sh' 'sed']
 ['rm' 'sh' 'sed' ... 'sh' 'sed' 'sed']]
arr.shape = (150, 100)
testUser = 31
arr = [['cat' 'mail' 'tcsh' ... 'sh' 'wc' 'sh']
 ['wc' 'sh' 'wc' ... 'wc' 'sh' 'wc']
 ['sh' 'wc' 'sh' ... 'sh' 'wc' 'sh']
 ...
 ['sh' 'date' 'sh' ... 'date' 'sh' 'date']
 ['sendmail' 'sendmail' 'sendmail' ... 'ftp' 'ls' 'mkdir']
 ['sh' 'date' 'sh' ... 'date' 'sh' 'date']]
arr.shape = (150, 100)
testUser = 32
arr = [['cpp' 'sh' 'xrdb' ... 'virtex' 'dvips' 'hostname']
 ['id' 'nawk' 'getopt' ... 'tput' 'mail' 'tput']
 ['sendmail' 'sendmail' 'sh' ... 'virtex' 'dvips' 'gs']
 ...
 ['mail' 'pq' 'pq' ... 'ex' 'ex' 'gs']
 ['ghostvie' 'more' 'mail' ... 'tput' 'finger' 'mail']
 ['sendmail' 'sendmail' 'sendmail' ... 'sendmail' 'sh' 'sendmail']]
arr.sh

(3) Feature Engineering

In [33]:
x_train = count_vec_usr = count_vec_all = []

number_of_users = 40
number_of_segments = 150
number_of_users_with_full_information = 10

for i in range(number_of_users*number_of_segments):
    #If we are still in the first 10 users information
    if(i < number_of_users_with_full_information*number_of_segments):
        count_vec_usr.append(' '.join(all_instructions[i,:]))
        count_vec_all.append(' '.join(all_instructions[i,:]))
    else:
        count_vec_all.append(' '.join(all_instructions[i,:]))

vectorizer = TfidfVectorizer(min_df=0.4,ngram_range=(1,2))
vectorizer.fit_transform(count_vec_all)

X_all = vectorizer.transform(count_vec_all)
X_all = X_all.toarray()
X_usr = vectorizer.transform(count_vec_usr)
X_usr = X_usr.toarray()

#should add also for first 10 users - the rest of the segments
unique = set()
for i in range(2000):
    for j in range(100):
        unique.add(all_instructions[i][j])
    x_train.append([random.randint(0, 2)])
    
count = 0
strange = []
unique_check = set()

for i in range(50,150):
    for j in range(100):
        unique_check.add(all_instructions[i][j])
        if (all_instructions[i][j] not in unique):
            count = count + 1
    strange.append([count])
    unique_check.clear()
    count = 0

(4) Model Fit & Predict

In [27]:
#x_test=x_train+strange
x_test=np.concatenate((x_train,strange))
x_test=np.c_[X_all,x_test]
x_train=np.c_[X_usr,x_train]
clf= LocalOutlierFactor(n_neighbors=25,contamination=0.1, novelty=True)
#clf = IsolationForest(behaviour='new', max_samples=20, contamination=0.5)
#clf=EllipticEnvelope(support_fraction=1.,contamination=0.1)
clf.fit(x_train)
t=clf.predict(x_test)
t[t == 1] = 0
t[t == -1] = 1
plt.show()
print(X_usr)

[[0.06312064 0.41187107 0.20737035 ... 0.15125476 0.         0.        ]
 [0.         0.43328658 0.21815271 ... 0.22276709 0.10104256 0.10104256]
 [0.0419673  0.2347222  0.11817879 ... 0.04022617 0.1459661  0.1459661 ]
 ...
 [0.03266644 0.24360355 0.12265041 ... 0.09393354 0.18936141 0.18936141]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.06549715 0.03297675 ... 0.03367427 0.         0.        ]]


(5) Push Results to Kaggle Challenge

In [ ]:
curr_user_results = t[50:150]
curr_user_results_df = pd.DataFrame(columns=["id","label"])

start_rows = 5000
till_rows = 5100

for i in range(curr_user_results.size):
    curr_user_results_df.loc[i] = ["User{}_{}-{}".format(testUser,start_rows,till_rows),curr_user_results[i]]
    start_rows = start_rows + 100
    till_rows = till_rows + 100
results_df = results_df.append(curr_user_results_df)

fname = "out/submission_results_{}.csv".format(date.today())
results_df.to_csv(fname, index=False)
#os.system('kaggle competitions submit -c detectingcyberattackscallenge -f "{0}" -m "{1}"'.format(fname, "Submitting results to kaggle"))